Bayesian Optimization & NN Opt
---

In [1]:
from bayes_opt import BayesianOptimization
from nnopt.nnopt import Optimizer
import time


In [2]:
def black_box_function(x, y):
    """Function with unknown internals we wish to maximize.

    This is just serving as an example, for all intents and
    purposes think of the internals of this function, i.e.: the process
    which generates its output values, as unknown.
    """
    return -x ** 2 - (y - 1) ** 2 + 1

In [3]:
Nopt = Optimizer(black_box_function, N=2, surrogate_hidden_layer=40, Rn=[(2, 4), (-3, 3)])

Bopt = BayesianOptimization(
    f=black_box_function,
    pbounds={'x': (2, 4), 'y': (-3, 3)},
    random_state=1,
)

Bayesian
---

In [4]:
timestamp = time.time()
Bopt.maximize(
    init_points=10,
    n_iter=40,
)

print("Bayesian took: {}".format(time.time() - timestamp))

|   iter    |  target   |     x     |     y     |
-------------------------------------------------
|  1        | -7.135    |  2.834    |  1.322    |
|  2        | -7.78     |  2.0      | -1.186    |
|  3        | -16.13    |  2.294    | -2.446    |
|  4        | -8.341    |  2.373    | -0.9266   |
|  5        | -7.392    |  2.794    |  0.2329   |
|  6        | -7.069    |  2.838    |  1.111    |
|  7        | -6.412    |  2.409    |  2.269    |
|  8        | -3.223    |  2.055    |  1.023    |
|  9        | -7.455    |  2.835    |  0.3521   |
|  10       | -12.11    |  2.281    | -1.811    |
|  11       | -3.071    |  2.0      |  1.266    |
|  12       | -3.671    |  2.151    |  1.211    |
|  13       | -3.017    |  2.0      |  1.13     |
|  14       | -3.036    |  2.0      |  0.809    |
|  15       | -3.211    |  2.0      |  0.5403   |
|  16       | -3.001    |  2.0      |  0.9615   |
|  17       | -3.001    |  2.0      |  1.035    |
|  18       | -3.0      |  2.0      |  0.9978   |


In [5]:
timestamp = time.time()
best, target = Nopt.run(random=10, optimization=40, fitting=100, verbose=True)

print("Best", best, "target", target)
print(black_box_function(*best))
print("NNopt took:", time.time() - timestamp)

sample [3.9024177 1.5430565] target -14.52377394040218
sample [ 2.0618443 -0.6086683] target -5.839015906481336
sample [ 2.4145553 -0.6857493] target -7.671828027596916
sample [3.1636212 1.1112328] target -9.020871742520796
sample [2.1675224  0.41651058] target -4.038613387326222
sample [3.085648  1.1384745] target -8.54039912652047
sample [2.0755537 1.7884398] target -3.929560217814526
sample [ 3.1486366  -0.62490606] target -11.554232023684392
sample [ 2.9852064 -1.5913081] target -14.626334801909536
sample [ 2.9101107 -2.2670088] target -18.142090734357282
Best random [2.0755537 1.7884398] -3.929560217814526
sample [1.8108096 1.8586013] target -3.0162276987201153
sample [1.2890996 2.151417 ] target -1.9875388589146752
sample [0.51930785 2.7519808 ] target -2.3391173038708644
sample [-0.27398095  3.5131197 ] target -5.390836172732542
sample [-0.90291846  4.159078  ] target -9.795036317521536
sample [-1.4299126  4.620575 ] target -14.153212926714332
sample [-1.9334027  4.828362 ] targ